# Notion Workspace Assistant with PraisonAIAgents

This notebook demonstrates how to interact with your Notion workspace using PraisonAIAgents in Google Colab. You can read, search, and update Notion pages with natural language queries, leveraging agentic tools for seamless document management—all within a notebook environment.

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Dhivya-Bharathy/PraisonAI/blob/main/examples/cookbooks/Programming_Code_Analysis_Agents/Notion_Workspace_Assistant.ipynb)


# Dependencies

In [ ]:
!pip install praisonaiagents openai requests --quiet

# Set Up Key

In [2]:
import os

# Set your Notion and OpenAI API keys here
NOTION_API_KEY = "ntn_.."  # <-- Replace with your actual Notion integration token
OPENAI_API_KEY = "sk-proj-_.."  # <-- Replace with your actual OpenAI API key

# Set environment variables for downstream libraries
os.environ["NOTION_API_KEY"] = NOTION_API_KEY
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

# Tools

In [3]:
from praisonaiagents import Agent
import requests
from IPython.display import display, Markdown

# Custom Tool

In [4]:
def notion_read_page_tool(page_id: str, notion_token: str) -> str:
    """
    Custom tool to read the content of a Notion page using the Notion API.
    """
    url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    headers = {
        "Authorization": f"Bearer {notion_token}",
        "Notion-Version": "2022-06-28"
    }
    response = requests.get(url, headers=headers)
    if response.ok:
        data = response.json()
        blocks = data.get("results", [])
        content = []
        for block in blocks:
            block_type = block.get("type", "")
            text = ""
            if block_type == "paragraph":
                text = block.get("paragraph", {}).get("text", [])
                if text:
                    text = text[0].get("plain_text", "")
            elif block_type == "heading_1":
                text = block.get("heading_1", {}).get("text", [])
                if text:
                    text = "# " + text[0].get("plain_text", "")
            elif block_type == "heading_2":
                text = block.get("heading_2", {}).get("text", [])
                if text:
                    text = "## " + text[0].get("plain_text", "")
            elif block_type == "heading_3":
                text = block.get("heading_3", {}).get("text", [])
                if text:
                    text = "### " + text[0].get("plain_text", "")
            if text:
                content.append(text)
        return "\n".join(content) if content else "No readable content found on this page."
    else:
        return f"Failed to fetch page content: {response.text}"

# YAML Prompt

In [5]:
yaml_prompt = """
agent_instructions: |
  You are a Notion workspace assistant. Help users read and manage their Notion pages.
  - Provide clear, organized summaries of page content.
  - Use the Notion API to fetch and display information.
  - If a page cannot be read, explain the issue clearly.

tools:
  - name: notion_read_page_tool
    description: >
      Reads the content of a Notion page using its page ID and the user's Notion integration token.
    parameters:
      - name: page_id
        type: string
        description: The Notion page ID to read.
      - name: notion_token
        type: string
        description: The user's Notion integration token.
"""
print(yaml_prompt)


agent_instructions: |
  You are a Notion workspace assistant. Help users read and manage their Notion pages.
  - Provide clear, organized summaries of page content.
  - Use the Notion API to fetch and display information.
  - If a page cannot be read, explain the issue clearly.

tools:
  - name: notion_read_page_tool
    description: >
      Reads the content of a Notion page using its page ID and the user's Notion integration token.
    parameters:
      - name: page_id
        type: string
        description: The Notion page ID to read.
      - name: notion_token
        type: string
        description: The user's Notion integration token.



# Main

In [15]:
# Define a wrapper to match the agent's expected tool signature
def notion_tool_wrapper(inputs):
    page_id = inputs.get("page_id", "")
    notion_token = inputs.get("notion_token", "")
    return notion_read_page_tool(page_id, notion_token)

# Create the agent and register the tool
agent = Agent(
    instructions="""
    You are a Notion workspace assistant. Help users read and manage their Notion pages.
    - Provide clear, organized summaries of page content.
    - Use the Notion API to fetch and display information.
    - If a page cannot be read, explain the issue clearly.
    """,
    tools={
        "notion_read_page_tool": notion_tool_wrapper
    }
)

# Call the tool directly (since agent.chat expects an LLM, but we want tool output)
response = agent.execute_tool("notion_read_page_tool", {
    "page_id": page_id,
    "notion_token": NOTION_API_KEY
})

# Check for error pattern and display a friendly message
if response.startswith("Failed to fetch page content:"):
    display(Markdown("### Notion Page Content\n*No content found or page is not accessible. Please check your page ID and sharing settings.*"))
else:
    display(Markdown(f"### Notion Page Content\n{response}"))

### Notion Page Content
*No content found or page is not accessible. Please check your page ID and sharing settings.*